In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [2]:
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [3]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

35178

In [4]:
def get_sentences(data):
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
    sentence_grouped = data.groupby("Sentence #").apply(agg_func)
    return [s for s in sentence_grouped] 

In [5]:
sentences = get_sentences(data)

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0: # word_prev, word_curr
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [7]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [8]:
X[1]

[{'bias': 1.0,
  'word.lower()': 'iranian',
  'word[-3:]': 'ian',
  'word[-2:]': 'an',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'JJ',
  'postag[:2]': 'JJ',
  'BOS': True,
  '+1:word.lower()': 'officials',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'NNS',
  '+1:postag[:2]': 'NN'},
 {'bias': 1.0,
  'word.lower()': 'officials',
  'word[-3:]': 'als',
  'word[-2:]': 'ls',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'NNS',
  'postag[:2]': 'NN',
  '-1:word.lower()': 'iranian',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'JJ',
  '-1:postag[:2]': 'JJ',
  '+1:word.lower()': 'say',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'VBP',
  '+1:postag[:2]': 'VB'},
 {'bias': 1.0,
  'word.lower()': 'say',
  'word[-3:]': 'say',
  'word[-2:]': 'ay',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit

In [8]:
from sklearn_crfsuite import CRF

crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100)

In [9]:
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5)


In [10]:
from sklearn_crfsuite.metrics import flat_classification_report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

              precision    recall  f1-score   support

       B-art       0.37      0.11      0.17       402
       B-eve       0.52      0.35      0.42       308
       B-geo       0.85      0.90      0.88     37644
       B-gpe       0.97      0.94      0.95     15870
       B-nat       0.66      0.37      0.47       201
       B-org       0.78      0.72      0.75     20143
       B-per       0.84      0.81      0.82     16990
       B-tim       0.93      0.88      0.90     20333
       I-art       0.11      0.03      0.04       297
       I-eve       0.34      0.21      0.26       253
       I-geo       0.82      0.79      0.80      7414
       I-gpe       0.92      0.55      0.69       198
       I-nat       0.61      0.27      0.38        51
       I-org       0.81      0.79      0.80     16784
       I-per       0.84      0.89      0.87     17251
       I-tim       0.83      0.76      0.80      6528
           O       0.99      0.99      0.99    887908

   micro avg       0.97   